In [19]:
import sys
sys.path.insert(0, '..')
from transformer.models import DecoderOnlyModel
from data_utils.Datasets import SerializedConcatDataset, PermSerializedConcatDataset, BinarySerializer
import pickle
import torch
import numpy as np

In [32]:
with open('serializer.pkl', 'rb') as inp:
    binser = pickle.load(inp)

# define model
vocab_size = binser.vocab_size
d_model = 512
num_heads = 8
num_layers = 8
d_ff = 512
max_seq_length = binser.max_seq_length
dropout = 0.3

transformer = DecoderOnlyModel(vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
transformer = transformer.to(dev)

saved_model_path = '../saved_models/perm_decoderOnly_serialized/perm_decoderOnly_serialized.pt'
transformer.load_state_dict(torch.load(saved_model_path), strict=False)

transformer.eval()

DecoderOnlyModel(
  (decoder_embedding): Embedding(30, 512)
  (positional_encoding): PositionalEncoding()
  (decoder_layers): ModuleList(
    (0-7): 8 x DecoderOnlyLayer(
      (self_attn): MultiHeadAttention(
        (W_q): Linear(in_features=512, out_features=512, bias=True)
        (W_k): Linear(in_features=512, out_features=512, bias=True)
        (W_v): Linear(in_features=512, out_features=512, bias=True)
        (W_o): Linear(in_features=512, out_features=512, bias=True)
      )
      (feed_forward): PositionWiseFeedForward(
        (fc1): Linear(in_features=512, out_features=512, bias=True)
        (fc2): Linear(in_features=512, out_features=512, bias=True)
        (relu): ReLU()
      )
      (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.3, inplace=False)
    )
  )
  (fc): Linear(in_features=512, out_features=30, bias=True)
  (dropout): Dropout(p=0.3, inplace=False

In [11]:
# load data to test
# load data
npz_path = '../data/augmented_and_padded_data.npz'
dataset = SerializedConcatDataset(npz_path, pad_to_length=max_seq_length)

In [46]:
x = dataset.__getitem__(0)
# find start harmonization index
idx = np.where(x == 4)[0][0]
print(idx)
y = np.pad(x[:idx+1], (x.shape[0]-idx-1, 0))
print(x.shape)
print(y.shape)
with np.printoptions(threshold=np.inf):
    print(y)

136
(1063,)
(1063,)
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0 

In [47]:
inp = torch.from_numpy( np.expand_dims(y[:-1], axis=0)).to(dev)
output = transformer(inp)

In [48]:
prediction = output.argmax(dim=2, keepdim=True).squeeze()
z = prediction.cpu().numpy()
with np.printoptions(threshold=np.inf):
    print(z)

[17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17
 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 12
 12 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17
 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17
 17 17 17 17 17 17 17 17 17 17 17 17 17 17 24 24 17 17 17 17 17 17 17 17
 17 17 17 17 17 17 17 17 17 12 12 12 12 17 17 17 17 17 17 17 17 17 17 17
 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17
 17 17 17 17 17 17 17 17 17 12 12 12 12 17 17 12 12 12 12 12 12 12 17 17
 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 12 12 12 12 12 12
 12 12 12 17 17 17 17 17 17 17 17 17 12 12 12 12 12 12 12 12 12 12 12 12
 12 12 12 12 12 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 17 12 12 12
 12 17 17 17 17 17 17 17 17 17 17 17 17 17 17 12 12 17 17 17 17 12 12 12
 12 12 12 12 17 17 17 17 17 12 12 12 12 12 12 12 17 17 17 17 17 17 17 17
 17 17 17 17 17 17 17 12 12 17 17 17 17 12 12 12 12